In [ ]:
# Из VECG нужна функция find_peaks, Тема надеюсь я правильно написал
# Я не все понимаю как должно быть, но надеюсь получилось выполнить просьбу
import VECG as ve
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import pywt

def cwt_spectrogram(x, fs, nNotes=12, detrend=True, normalize=True):
    if detrend:
        x = scipy.signal.detrend(x)
    if normalize:
        x = (x - np.mean(x)) / np.std(x)
    N = len(x)
    dt = 1.0/fs
    time = np.arange(N) * dt
    nOctaves = int(np.log2(2*np.floor(N/2.0)))
    scales = 2**np.arange(1, nOctaves, 1.0/nNotes)
    cwtmatr, freqs = pywt.cwt(x, scales, 'morl')
    return time, cwtmatr, freqs

def GetSpectr(Numb1,Namb2,ecgSig):
    # Нашли пики для дальнейшего выведения циклов
    # Если пользовотель ввел не то значение (столько циклов нет)
    # Он ничего не получит
    Ecgpeaks = ve.find_peaks()
    if Numb2 > len(Ecgpeaks):
        print('В ЭКГ сигнале нет столько циклов')
        return None
    else:
        # +1 здесь чтобы не исключать значение зубца R под номером Numb2
        # Так как мы берем со значения Numb1 до значения Numb2 ВКЛЮЧИТЕЛЬНО
        ecgSigcrop = ecgSig[Ecgpeaks[Numb1-1]:(Ecgpeaks[Numb2-1]+1)]
        fs = 1
        # Вычисление спектрограммы с помощью CWT
        t,cwtmatr, freqs = cwt_spectrogram(ecgSigcrop, fs, nNotes = 24)
        freqs = freqs*1000
        plt.subplot(2, 1, 2)
        plt.imshow(np.abs(cwtmatr), extent=[0, 2 , freqs[-1], freqs[0]], aspect='auto', cmap='jet')
        plt.colorbar(label='Амплитуда')
        plt.title('Спектрограмма CWT')
        plt.xlabel('Время (с)')
        plt.ylabel('Частота (Гц)')
        plt.show()